In this notebook we explain the function get_data_merged_dynamic in the classe Learningset

In [1]:
%%time
from transplant.tools.learningset import Learningset

learningset = Learningset()

X_train, X_test, y_train, y_test, X_col = learningset.get_data_merged_dynamic()

C:\Users\Delanoue\Documents\GitHubRepo\batch_5_transplant\transplant\tools\learningset.py:126: FutureWarning: 'id_patient' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  mean_dynamic_train = train_dynamic_0.groupby(['id_patient']).mean().mean()
C:\Users\Delanoue\Documents\GitHubRepo\batch_5_transplant\transplant\tools\learningset.py:132: FutureWarning: 'id_patient' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  grouped_time = X_dyn.groupby(['id_patient'])['time']
C:\Users\Delanoue\Documents\GitHubRepo\batch_5_transplant\transplant\tools\learningset.py:157: FutureWarning: 'id_patient' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  grouped = X_dyn.groupby(['id_patient'])


Wall time: 13.7 s


# Full Time Series -- Data Preparation 

In [2]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd
import numpy as np

#from datetime import datetime

## Get Static

In [5]:
from transplant.tools.dataset import Dataset

dataset = Dataset()

In [6]:
from transplant.tools.learningset import Learningset

learningset = Learningset()

train_static_0, test_static_0 = learningset.get_static_filled()

## Get Static

In [7]:
train_dynamic_0, test_dynamic_0 = dataset.get_dynamic()

We need to extract 3 static information from this time series : 

    -When the operation started
    -When it ended
    -How long it was

## Function to get the beginning, end and length of operation 

In [8]:
mean_dynamic_train = train_dynamic_0.groupby(['id_patient']).mean().mean()

In [9]:
train_dynamic_0 = train_dynamic_0.fillna(mean_dynamic_train)
test_dynamic_0 = test_dynamic_0.fillna(mean_dynamic_train)

In [10]:
def add_start_end_length_op_to_static(X_stat, X_dyn):
    grouped_time = X_dyn.groupby(['id_patient'])['time']

    time_start_df = grouped_time.first().to_frame()
    time_start_df.columns = ['start_operation']
    time_start_df['id_patient'] = time_start_df.index

    X_return = pd.merge(X_stat, time_start_df, on='id_patient')

    time_ends_df = grouped_time.last().to_frame()
    time_ends_df.columns = ['ends_operation']
    time_ends_df['id_patient'] = time_ends_df.index

    X_return = pd.merge(X_return, time_ends_df, on='id_patient')

    X_return['length_op'] = X_return['ends_operation'] - \
        X_return['start_operation']

    return X_return

In [11]:
train_static_1 = add_start_end_length_op_to_static(
    train_static_0, train_dynamic_0)
test_static_1 = add_start_end_length_op_to_static(
    test_static_0, test_dynamic_0)

In [17]:
type(train_static_1['start_operation'])

pandas.core.series.Series

## Function to obtain the full time series in numpy array

In [12]:
def get_timeseries_in_array(X_stat, X_dyn, dyn_to_drop=['id_patient', 'time']):

    X_return = X_stat

    grouped = X_dyn.groupby(['id_patient'])

    list_time_serie_col = X_dyn.drop(dyn_to_drop, axis=1).columns

    for i in list_time_serie_col:
        df_muette = grouped[i].apply(np.array).to_frame()
        df_muette['id_patient'] = df_muette.index
        X_return = pd.merge(X_return, df_muette, on='id_patient')

    return X_return

In [13]:
train_merged_dyn_0 = get_timeseries_in_array(train_static_1, train_dynamic_0)
test_merged_dyn_0 = get_timeseries_in_array(test_static_1, test_dynamic_0)

In [14]:
train_merged_dyn_0.head()


,Age_donor,Aspirations_donor,BMI_donor,Donneur_CPT,Insuffisance_renale,LAS,PAPS,PFO,PF_donor,Poids,...,PNIm,PNIs,Pmax,Pmean,Pplat,RR(co2),SpO2,SvO2 (m),Temp,VT
0,41,3.0,25.689562,5550.0,0.0,39.3,68.884058,0.0,457.0,43.0,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 98.0, 98.0, 98.0, 99.0, 97.0, 97.0, 98.0...","[0.0, 0.0, 81.0, 0.0, 70.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 115.0, 0.0, 102.0, 0.0, 0.0, 0.0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[374.0, 373.0, 374.0, 374.0, 374.0, 374.0, 374...","[73.54861055381699, 73.54861055381699, 73.5486...","[74.27279525349674, 74.27279525349674, 74.2727...","[89.0338888768584, 89.0338888768584, 89.033888...","[86.52837409798167, 86.52837409798167, 86.5283..."
1,43,2.0,19.531250,4750.0,0.0,31.7,68.884058,0.0,497.0,51.0,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[39.11492054010727, 39.11492054010727, 39.1149...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[74.27279525349674, 74.27279525349674, 74.2727...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,51,3.0,28.685145,5250.0,0.0,51.9,55.000000,0.0,240.0,55.0,...,"[37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37....","[0.0, 0.0, 0.0, 82.0, 0.0, 85.0, 0.0, 0.0, 81....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[36.0, 36.0, 36.0, 36.0, 36.0, 36.0, 36.0, 36....","[39.11492054010727, 39.11492054010727, 39.1149...","[0.0, 0.0, 0.0, 99.0, 0.0, 101.0, 0.0, 0.0, 98...","[0.0, 0.0, 0.0, 123.0, 0.0, 126.0, 0.0, 0.0, 1...","[74.27279525349674, 74.27279525349674, 74.2727...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[100.0, 100.0, 100.0, 99.0, 100.0, 99.0, 98.0,..."
3,34,3.0,31.960227,7000.0,1.0,36.3,38.000000,0.0,152.0,65.0,...,"[0.0, 126.0, 0.0, 0.0, 98.0, 0.0, 0.0, 0.0, 0....","[0.0, 150.0, 0.0, 0.0, 124.0, 0.0, 0.0, 0.0, 0...","[2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[95.0, 97.0, 97.0, 97.0, 92.0, 94.0, 94.0, 98....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,63,0.0,22.892820,4450.0,0.0,37.4,40.000000,0.0,390.0,70.0,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[39.11492054010727, 39.11492054010727, 39.1149...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[74.27279525349674, 74.27279525349674, 74.2727...","[89.0338888768584, 89.0338888768584, 89.033888...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
